# Part 1: Data cleaning 


In [ ]:
#import initial libraries

import pandas as pd
import numpy as np


In [ ]:
#import data 

df = pd.read_csv('data/abrams.csv')

In [ ]:
# look at basic info about data

df.info()
# this data set consists of 6824 Tweets
# Twarc search for keyword "Twitter", the morning after Democratic victories in Georgia Senate races 
# Stacey Abrams was widely credited with bringing about Georgia transition from red to blue 

In [ ]:
# count number of languages in data set

count_lang = df['lang'].unique()
print(len(count_lang), count_lang)

In [ ]:
# tweets are in 23 different languages

# I'll be working only with Tweets in English
# drop tweets in all other languages
# now working with 6659 Tweets 

df = df[df.lang == 'en']
df.shape

In [ ]:
# drop unnecessary columns 

df = df.drop(['tweet_url', 'created_at', 'media', 'urls','in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_or_quote_id',
       'retweet_or_quote_screen_name', 'retweet_or_quote_user_id', 'source',
       'user_created_at', 'user_name', 'user_verified', 'user_friends_count', 'user_listed_count',
       'user_statuses_count', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'coordinates', 'lang', 'user_location', 'user_time_zone', 'user_urls', 'place'], axis=1)

In [ ]:
# check start time & date of data

df.iloc[0]

# first Tweet downloaded Jan 6, 2021 at 18:42:31

In [ ]:
# check end time & date of data 

df.iloc[-1]

# last Tweet on Jan 6, 2021 at 19:02:01

# Part 2: Text processing for NLP 

In [ ]:
# create variable for "text" column 

text = df['text'] 

In [ ]:
# tokenize, remove stopwords, remove urls, lowercase, remove punctuation, remove numbers
# import necessary libraries: ntlk etc.

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 

stop = stopwords.words('english')

punc = list(set(string.punctuation))

def tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r'', text)

def process_text(text):
    text = remove_url(text)
    text = tokenizer(text)
    text = [word.lower() for word in text]
    text = [re.sub('[0-9]+', '', word) for word in text]
    text = [word for word in text if word not in punc]
    text = [word for word in text if word not in stop]
    text = [each for each in text if len(each) > 1]
    text = [word for word in text if ' ' not in word]
     
    return text

In [ ]:
# apply text processing functions to text

df['processed_text'] = df['text'].apply(process_text)

In [ ]:
# look at some of processed text

pd.set_option('display.max_colwidth', -1)
df['processed_text'][:20]

In [ ]:
# part-of-speech tagging 

ready_for_pos = df['processed_text']

def pos_tagging(text):
    pos_tag = [pos_tag(word) for word in ready_for_pos]

df['pos_tagged'] = df.processed_text.apply(lambda x: pos_tag(x))

In [ ]:
# lemmatizing

pos_tagged = df['pos_tagged']

wordnet = WordNetLemmatizer() 

lemmatized = [[wordnet.lemmatize(word[0]) for word in words] for words in pos_tagged]

In [ ]:
# look at lemmatized text

df['lemmatized'] = lemmatized
lemmatized[:20]

In [ ]:
# before vectorizing, cast lists of words back into strings

df['final_docs'] = df['lemmatized'].apply(lambda x: " ".join(x))
pd.set_option('display.max_colwidth', -1)
final_docs = df['final_docs']
final_docs[3000:3020]

In [ ]:
#create document term matrix with TFIDF

#import vectorizing tool (usee TFIDF)
from sklearn.feature_extraction.text import TfidfVectorizer
# set max_features to 2000 (specifies the number of most frequently occurring words for which we want to create feature vectors)
# set min_df to 5 (word must occur in at least 5 documents)
# set max_df to 0.85 (word must not occur in more than 85 percent of the documents) 

tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.85, ngram_range=(1, 2), stop_words='english')  
doc_term_matrix_1 = tfidfconverter.fit_transform(df['final_docs'].values.astype('U'))

# Part 3: run NMF and LDA models, for topic modeling

In [ ]:
#run NMF model 

#import NMF tool 
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=6)
nmf_Z = nmf_model.fit_transform(doc_term_matrix_1)


In [ ]:
# run LDA model

#import LDA tool 
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components = 6, max_iter=10, learning_method='online', learning_decay=.9)
lda_Z = lda_model.fit_transform(doc_term_matrix_1)

In [ ]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, tfidfconverter )
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, tfidfconverter )
print("=" * 20)
 

# Part 4: Run visualization and testing of LDA model

In [24]:
# test LDA model

# log likelihood (higher score is better)
print("Log likelihood: ", lda_model.score(doc_term_matrix_1))

# perplexity (lower score is better)
print("Perplexity: ", lda_model.perplexity(doc_term_matrix_1))

Log likelihood:  -167849.4323281318
Perplexity:  920.0496458340432


In [25]:
# visualization of LDA model 
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      1.465019  -47.625065  1       1        18.647875
4      54.854797 -36.476795  2       1        17.844918
2      32.212929 -61.444572  3       1        17.625091
3      5.111096  -14.110332  4       1        16.537567
0      26.350170 -33.374702  5       1        15.195713
1      38.109818 -7.222781   6       1        14.148837, topic_info=                    Term        Freq       Total Category  logprob  loglift
1800  tweet               121.000000  121.000000  Default  30.0000  30.0000
306   carrying            108.000000  108.000000  Default  29.0000  29.0000
87    abrams tweet        112.000000  112.000000  Default  28.0000  28.0000
13    abrams carrying     107.000000  107.000000  Default  27.0000  27.0000
307   carrying democracy  107.000000  107.000000  Default  26.0000  26.0000
...                  ...         ...         ...      ...      ...      ...
1931  woman               27.906480   101.927980  Topic6  -4.8259   0.6601 
753   governor            24.027392   58.261778   Topic6  -4.9756   1.0698 
1535  senate              23.525733   73.764181   Topic6  -4.9967   0.8128 
1647  state               26.803888   194.516245  Topic6  -4.8662  -0.0264 
731   going               19.129909   34.845148   Topic6  -5.2035   1.3559 

[307 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
5     1      0.923816  abrams american     
13    4      0.992341  abrams carrying     
15    6      0.971817  abrams cheated      
17    6      0.983391  abrams consequential
33    5      0.985005  abrams goddess      
...  ..           ...                   ...
1985  2      0.080760  year                
1985  4      0.020190  year                
1985  5      0.080760  year                
1985  6      0.323039  year                
1990  6      0.958653  year torched        

[422 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 3, 4, 1, 2])

In [26]:
# cross-validation to find best parameters for LDA model

#import cross-validation tool
from sklearn.model_selection import GridSearchCV

# define search parameters
search_params = {'n_components': [5, 8, 10, 12], 'learning_decay': [.5, .7, .9]}

# initialize model for cross-validation
lda = LatentDirichletAllocation()

# initialize grid search class 
model = GridSearchCV(lda, param_grid=search_params)

# run grid search 
model.fit(doc_term_matrix_1)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 8, 10, 12]})

In [27]:
# what is best model? 
best_lda_model = model.best_estimator_

# print parameters for best model
print("Parameters for best model: ", model.best_params_)

# print log likelihood score
print("Log likelihood score of best model: ", model.best_score_)

# print perplexity score
print("Perplexity score of best model: ", best_lda_model.perplexity(doc_term_matrix_1))

Parameters for best model:  {'learning_decay': 0.7, 'n_components': 5}
Log likelihood score of best model:  -39634.21593541675
Perplexity score of best model:  1027.9415668034726


In [28]:
# top topics of 2nd LDA model

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model_2, tfidfconverter )
print("=" * 20)

LDA Model:


NameError: name 'lda_model_2' is not defined

In [ ]:
# 2nd LDA model: running new "best model" parameters

lda_model_2 = LatentDirichletAllocation(n_components = 5, max_iter=10, learning_method='online', learning_decay=0.5)
lda_Z_2 = lda_model_2.fit_transform(doc_term_matrix_1)

# visualization of 2nd LDA model 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_2, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_2, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

# Part 5: Additional data analysis

In [ ]:
# insert NMF model, LDA model 1, and LDA model 2 results back into dataframe
topic_values = nmf_model.transform(doc_term_matrix_1)
df['NMF_topic'] = topic_values.argmax(axis=1)
df.head(1)

In [ ]:
topic_values = lda_model.transform(doc_term_matrix_1)
df['LDA1_topic'] = topic_values.argmax(axis=1)
df.head(1)

In [ ]:
topic_values = lda_model_2.transform(doc_term_matrix_1)
df['LDA2_topic'] = topic_values.argmax(axis=1)
df.head(20)

In [ ]:
df.tail(20)

In [ ]:
df.columns


In [ ]:
# drop unnecessary columns
df = df.drop(['id', 'parsed_created_at', 'user_screen_name', 'text',
       'hashtags', 'user_id', 'processed_text', 'pos_tagged', 'lemmatized', 'final_docs'], axis=1)

In [ ]:
df.head()

